# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
# YOUR CHANGES HERE

from datetime import datetime
import pandas as pd

TICKERS = ["AAPL","MSFT","GOOGL","AMZN","NVDA","META"]
RELEASE_DATE = "2025-09-01"

import yfinance as yf

rel_ts = pd.to_datetime(RELEASE_DATE).normalize()
# month end strictly before release date (calendar month end)
month_end_before_release = (rel_ts - pd.offsets.MonthBegin(1)) - pd.offsets.Day(1)
# fetch a wide window, we'll trim later
start = (month_end_before_release - pd.DateOffset(months=36)).normalize()
end = rel_ts

# Ask yfinance to return prices already adjusted for splits/dividends
# -> This means "Close" is adjusted close and there is NO "Adj Close" column at all.
raw = yf.download(
    TICKERS,
    start=start.strftime("%Y-%m-%d"),
    end=end.strftime("%Y-%m-%d"),
    auto_adjust=True,           # <-- key change
    progress=False,
    threads=True
)

if raw is None or raw.empty:
    raise ValueError("yfinance returned no data. Check tickers, dates, or network connection.")

# Handle single vs multi-ticker shapes, and drop tickers with all-NaN data
if isinstance(raw.columns, pd.MultiIndex):
    # Level 0 is field name; Level 1 is ticker
    if "Close" not in raw.columns.get_level_values(0):
        raise KeyError(f"'Close' not found in columns: {sorted(set(raw.columns.get_level_values(0)))}")
    data = raw["Close"].copy()  # shape: (dates, tickers)
else:
    # Single ticker → flat columns
    if "Close" not in raw.columns:
        raise KeyError(f"'Close' not in columns: {list(raw.columns)}")
    # convert to a (dates x tickers) DataFrame
    one_ticker_name = TICKERS[0] if len(TICKERS) == 1 else TICKERS[0]
    data = raw[["Close"]].rename(columns={"Close": one_ticker_name})

# Drop columns that are entirely NA (e.g., delisted tickers or bad symbols)
data = data.dropna(axis=1, how="all")
missing = [t for t in TICKERS if t not in data.columns]
if missing:
    print(f"Warning: no data for these tickers and they will be excluded: {missing}")

Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [2]:
# YOUR CHANGES HERE

# Trim to month strictly before release date
data = data.loc[:month_end_before_release]

# Take last trading day of each calendar month
monthly_last = data.resample("M").last()

# Ensure we have at least 24 monthly rows
if len(monthly_last) < 24:
    raise ValueError(f"Only {len(monthly_last)} monthly rows available; need >= 24. "
                     f"Try earlier RELEASE_DATE or different tickers.")

historical_prices = monthly_last.tail(24).copy()
historical_prices.index.name = "date"
historical_prices = historical_prices.reset_index()
historical_prices["date"] = pd.to_datetime(historical_prices["date"]).dt.strftime("%Y-%m-%d")

# Keep only up to 6 columns actually present (some might be missing)
cols = ["date"] + [c for c in TICKERS if c in historical_prices.columns]
historical_prices = historical_prices[cols]

historical_prices.to_csv("historical_prices.tsv", sep="\t", index=False)
print("Wrote historical_prices.tsv with shape:", historical_prices.shape)
historical_prices.head()

Wrote historical_prices.tsv with shape: (24, 7)


/tmp/ipykernel_2292/4021044140.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_last = data.resample("M").last()


Ticker,date,AAPL,MSFT,GOOGL,AMZN,NVDA,META
0,2023-08-31,186.047653,322.894012,135.358215,138.009995,49.324642,294.275604
1,2023-09-30,169.549286,311.062317,130.079880,127.120003,43.475826,298.572021
2,2023-10-31,169.113541,333.090332,123.340286,133.089996,40.758278,299.626221
3,2023-11-30,188.355301,374.042267,131.739899,146.089996,46.745087,325.365021
4,2023-12-31,190.913651,371.209106,138.857224,151.940002,49.499973,352.028717


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [3]:
# YOUR CHANGES HERE

import pandas as pd

prices = pd.read_csv("historical_prices.tsv", sep="\t", parse_dates=["date"])
tickers = [c for c in prices.columns if c != "date"]

rets = prices.copy()
rets[tickers] = prices[tickers].pct_change()
rets = rets.dropna().reset_index(drop=True)

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [4]:
# YOUR CHANGES HERE

rets["date"] = rets["date"].dt.strftime("%Y-%m-%d")
rets.to_csv("historical_returns.tsv", sep="\t", index=False)
print("Wrote historical_returns.tsv with shape:", rets.shape)
rets.head()

Wrote historical_returns.tsv with shape: (23, 7)


,date,AAPL,MSFT,GOOGL,AMZN,NVDA,META
0,2023-09-30,-0.088678,-0.036643,-0.038995,-0.078907,-0.118578,0.014600
1,2023-10-31,-0.002570,0.070815,-0.051811,0.046963,-0.062507,0.003531
2,2023-11-30,0.113780,0.122945,0.068101,0.097678,0.146886,0.085903
3,2023-12-31,0.013583,-0.007574,0.054026,0.040044,0.058934,0.081950
4,2024-01-31,-0.042227,0.057281,0.002935,0.021456,0.242418,0.102215


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [5]:
# YOUR CHANGES HERE

rets = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])
tickers = [c for c in rets.columns if c != "date"]

est_returns = pd.DataFrame({
    "asset": tickers,
    "estimated_return": rets[tickers].mean().values
}).sort_values("asset").reset_index(drop=True)

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [6]:
# YOUR CHANGES HERE

est_returns.to_csv("estimated_returns.tsv", sep="\t", index=False)
print("Wrote estimated_returns.tsv")
est_returns

Wrote estimated_returns.tsv


,asset,estimated_return
0,AAPL,0.006385
1,AMZN,0.025579
2,GOOGL,0.017641
3,META,0.047035
4,MSFT,0.023842
5,NVDA,0.064383


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [7]:
# YOUR CHANGES HERE

import numpy as np

rets = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])
tickers = [c for c in rets.columns if c != "date"]

cov = np.cov(rets[tickers].T, ddof=1)
cov_df = (
    pd.DataFrame(cov, index=tickers, columns=tickers)
    .rename_axis("asset_i")
    .reset_index()
    .melt(id_vars="asset_i", var_name="asset_j", value_name="covariance")
)

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [8]:
# YOUR CHANGES HERE

cov_df.to_csv("estimated_covariance.tsv", sep="\t", index=False)
print("Wrote estimated_covariance.tsv")
cov_df.head()

Wrote estimated_covariance.tsv


,asset_i,asset_j,covariance
0,AAPL,AAPL,0.003540
1,MSFT,AAPL,0.001019
2,GOOGL,AAPL,0.000595
3,AMZN,AAPL,0.001168
4,NVDA,AAPL,0.002231


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [9]:
# YOUR CHANGES HERE

est = pd.read_csv("estimated_returns.tsv", sep="\t")
best_idx = est["estimated_return"].idxmax()
alloc = pd.Series(0.0, index=est["asset"])
alloc.iloc[best_idx] = 1.0

Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [10]:
# YOUR CHANGES HERE

maxret_df = pd.DataFrame({"asset": est["asset"], "allocation": alloc.values})
assert np.isclose(maxret_df["allocation"].sum(), 1.0)
maxret_df.to_csv("maximum_return.tsv", sep="\t", index=False)
print("Wrote maximum_return.tsv")
maxret_df

Wrote maximum_return.tsv


,asset,allocation
0,AAPL,0.0
1,AMZN,0.0
2,GOOGL,0.0
3,META,0.0
4,MSFT,0.0
5,NVDA,1.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [19]:
# YOUR CHANGES HERE

from scipy.optimize import minimize


cov_df = pd.read_csv("estimated_covariance.tsv", sep="\t")
assets = sorted(cov_df["asset_i"].unique().tolist())
n = len(assets)
Sigma = cov_df.pivot(index="asset_i", columns="asset_j", values="covariance").loc[assets, assets].values

def portfolio_variance(w, S):
    return float(w @ S @ w)

cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
bounds = [(0.0, 1.0)] * n
w0 = np.full(n, 1.0/n)

res = minimize(portfolio_variance, w0, args=(Sigma,), method="SLSQP", bounds=bounds, constraints=cons)
if not res.success:
    raise RuntimeError(res.message)

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [20]:
# YOUR CHANGES HERE

w_minrisk = res.x / res.x.sum()
minrisk_df = pd.DataFrame({"asset": assets, "allocation": w_minrisk})
minrisk_df.to_csv("minimum_risk.tsv", sep="\t", index=False)
print("Wrote minimum_risk.tsv")
minrisk_df

Wrote minimum_risk.tsv


,asset,allocation
0,AAPL,4.206671e-01
1,AMZN,1.006495e-01
2,GOOGL,2.693734e-01
3,META,1.387779e-17
4,MSFT,2.093101e-01
5,NVDA,0.000000e+00


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [21]:
# YOUR CHANGES HERE

mu_df = pd.read_csv("estimated_returns.tsv", sep="\t")
assets = mu_df["asset"].tolist()
mu = mu_df.set_index("asset")["estimated_return"].loc[assets].values
cov_df = pd.read_csv("estimated_covariance.tsv", sep="\t")
Sigma = cov_df.pivot(index="asset_i", columns="asset_j", values="covariance").loc[assets, assets].values

def solve_min_var_for_target(mu, Sigma, R_target):
    n = len(mu)
    def var_obj(w): return float(w @ Sigma @ w)
    cons = [
        {"type":"eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type":"eq", "fun": lambda w: float(mu @ w) - R_target},
    ]
    bounds = [(0.0, 1.0)] * n
    w0 = np.full(n, 1.0/n)
    res = minimize(var_obj, w0, method="SLSQP", bounds=bounds, constraints=cons)
    if not res.success:
        raise RuntimeError(res.message)
    return res.x / res.x.sum()

minrisk_alloc = pd.read_csv("minimum_risk.tsv", sep="\t").set_index("asset").loc[assets]["allocation"].values
maxret_alloc  = pd.read_csv("maximum_return.tsv", sep="\t").set_index("asset").loc[assets]["allocation"].values
R_min, R_max = mu @ minrisk_alloc, mu @ maxret_alloc

targets = np.linspace(R_min, R_max, 101)
rows = []
for k, R_tgt in enumerate(targets):
    w = solve_min_var_for_target(mu, Sigma, R_tgt)
    rows.append({
        "portfolio_id": k,
        "target_return": R_tgt,
        "realized_return": float(mu @ w),
        "risk": float(np.sqrt(w @ Sigma @ w)),
        **{f"alloc_{a}": wi for a, wi in zip(assets, w)}
    })

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [22]:
# YOUR CHANGES HERE

ef = pd.DataFrame(rows)
ef.to_csv("efficient_frontier.tsv", sep="\t", index=False)
print("Wrote efficient_frontier.tsv with shape:", ef.shape)
ef.head()

Wrote efficient_frontier.tsv with shape: (101, 10)


,portfolio_id,target_return,realized_return,risk,alloc_AAPL,alloc_AMZN,alloc_GOOGL,alloc_META,alloc_MSFT,alloc_NVDA
0,0,0.015003,0.015003,0.045916,0.427792,0.142367,0.261002,0.000000e+00,0.168838,5.551115e-17
1,1,0.015496,0.015496,0.046088,0.401634,0.158131,0.259428,0.000000e+00,0.180807,2.775558e-17
2,2,0.015990,0.015990,0.046336,0.375475,0.173894,0.257854,2.775558e-17,0.192777,0.000000e+00
3,3,0.016484,0.016484,0.045165,0.459895,0.000000,0.205507,8.386268e-02,0.250734,6.865894e-20
4,4,0.016978,0.016978,0.045258,0.448815,0.000000,0.204859,9.663985e-02,0.249687,0.000000e+00


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [23]:
# YOUR CHANGES HERE

mu_df = pd.read_csv("estimated_returns.tsv", sep="\t").set_index("asset")
cov_df = pd.read_csv("estimated_covariance.tsv", sep="\t")
assets = mu_df.index.tolist()
Sigma = cov_df.pivot(index="asset_i", columns="asset_j", values="covariance").loc[assets, assets].values
mu = mu_df.loc[assets, "estimated_return"].values

rng = np.random.default_rng(704)
num_trials = 1000
T = 23

wins = dict.fromkeys(assets, 0)
for _ in range(num_trials):
    samples = rng.multivariate_normal(mean=mu, cov=Sigma, size=T)
    means = samples.mean(axis=0)
    winner = assets[int(np.argmax(means))]
    wins[winner] += 1

Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [24]:
# YOUR CHANGES HERE

probs = pd.DataFrame({
    "asset": assets,
    "probability": [wins[a]/num_trials for a in assets]
})
probs.to_csv("max_return_probabilities.tsv", sep="\t", index=False)
print("Wrote max_return_probabilities.tsv")
probs

Wrote max_return_probabilities.tsv


,asset,probability
0,AAPL,0.002
1,AMZN,0.007
2,GOOGL,0.012
3,META,0.196
4,MSFT,0.006
5,NVDA,0.777


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.